In [1]:
import pandas as pd
import numpy as np

# ============================================================
# LOAD YOUR DATA
# ============================================================
EQUITY_FILE = "./13-trading_output_regression_insp500_spyfilter_cap15/13-equity_curve_regression_insp500_spyfilter_cap15.parquet"

equity = pd.read_parquet(EQUITY_FILE)
equity["date"] = pd.to_datetime(equity["date"])
equity = equity.sort_values("date").reset_index(drop=True)

# ============================================================
# OPTION 1: Use pandas-datareader to fetch from FRED
# ============================================================
try:
    import pandas_datareader.data as web
    
    start = equity["date"].min()
    end = equity["date"].max()
    
    # 3-Month T-Bill (secondary market rate, daily)
    tbill = web.DataReader("DTB3", "fred", start, end)
    tbill.columns = ["tbill_rate"]
    tbill = tbill.reset_index().rename(columns={"DATE": "date"})
    tbill["date"] = pd.to_datetime(tbill["date"])
    
    # Rate is annualized %, convert to daily decimal
    tbill["tbill_daily"] = (tbill["tbill_rate"] / 100) / 252
    
    print("Loaded T-Bill data from FRED")
    print(f"Date range: {tbill['date'].min()} to {tbill['date'].max()}")
    print(f"Average annual rate: {tbill['tbill_rate'].mean():.2f}%")
    
except ImportError:
    print("pandas-datareader not installed. Using manual approach.")
    tbill = None

# ============================================================
# OPTION 2: Manual Historical Averages (if no datareader)
# ============================================================
# Average 3-month T-Bill rates by period:
# TBILL_RATES_ANNUAL = {
#     1999: 4.66, 2000: 5.85, 2001: 3.45, 2002: 1.62, 2003: 1.02,
#     2004: 1.38, 2005: 3.16, 2006: 4.73, 2007: 4.48, 2008: 1.37,
#     2009: 0.15, 2010: 0.14, 2011: 0.05, 2012: 0.09, 2013: 0.06,
#     2014: 0.03, 2015: 0.05, 2016: 0.32, 2017: 0.93, 2018: 1.94,
#     2019: 2.07, 2020: 0.36, 2021: 0.04, 2022: 1.94, 2023: 5.03,
#     2024: 5.25, 2025: 4.50,  # estimate
# }

# ============================================================
# CALCULATE CASH RETURN CONTRIBUTION
# ============================================================
equity["year"] = equity["date"].dt.year
equity["cash_pct"] = equity["cash"] / equity["portfolio_value"]

# Get daily T-Bill rate
if tbill is not None:
    equity = equity.merge(tbill[["date", "tbill_daily"]], on="date", how="left")
    equity["tbill_daily"] = equity["tbill_daily"].ffill().fillna(0)
else:
    # Use annual averages
    equity["tbill_annual"] = equity["year"].map(TBILL_RATES_ANNUAL).fillna(0)
    equity["tbill_daily"] = (equity["tbill_annual"] / 100) / 252

# Calculate daily cash return
equity["cash_return_daily"] = equity["cash_pct"].shift(1) * equity["tbill_daily"]
equity["cash_return_daily"] = equity["cash_return_daily"].fillna(0)

# ============================================================
# RECONSTRUCT PORTFOLIO WITH CASH RETURNS
# ============================================================

# Original strategy return
equity["strat_ret"] = equity["portfolio_value"].pct_change().fillna(0)

# Adjusted return = original return + cash return
equity["strat_ret_with_cash"] = equity["strat_ret"] + equity["cash_return_daily"]

# Build adjusted equity curve
equity["portfolio_with_cash"] = (1 + equity["strat_ret_with_cash"]).cumprod()
equity["portfolio_with_cash"] = equity["portfolio_with_cash"] * equity["portfolio_value"].iloc[0]

# ============================================================
# COMPARE RESULTS
# ============================================================
def calc_cagr(series):
    total_ret = series.iloc[-1] / series.iloc[0]
    years = len(series) / 252
    return total_ret ** (1/years) - 1

def calc_sharpe(returns):
    return np.sqrt(252) * returns.mean() / returns.std()

original_cagr = calc_cagr(equity["portfolio_value"])
adjusted_cagr = calc_cagr(equity["portfolio_with_cash"])
cash_boost = adjusted_cagr - original_cagr

original_sharpe = calc_sharpe(equity["strat_ret"])
adjusted_sharpe = calc_sharpe(equity["strat_ret_with_cash"])

print("\n" + "="*60)
print(" CASH RETURN IMPACT ANALYSIS")
print("="*60)

print(f"\n  Original CAGR (0% on cash):     {original_cagr*100:.2f}%")
print(f"  Adjusted CAGR (T-Bill on cash): {adjusted_cagr*100:.2f}%")
print(f"  Cash Return Boost:              +{cash_boost*100:.2f}%")

print(f"\n  Original Sharpe:                {original_sharpe:.3f}")
print(f"  Adjusted Sharpe:                {adjusted_sharpe:.3f}")

# ============================================================
# YEARLY BREAKDOWN
# ============================================================
print("\n" + "-"*60)
print(" YEARLY CASH RETURN CONTRIBUTION")
print("-"*60)

yearly = equity.groupby("year").agg({
    "cash_pct": "mean",
    "tbill_daily": lambda x: x.mean() * 252 * 100,  # annualize
    "cash_return_daily": lambda x: x.sum() * 100,   # total contribution
    "strat_ret": lambda x: ((1+x).prod() - 1) * 100,
    "strat_ret_with_cash": lambda x: ((1+x).prod() - 1) * 100,
}).round(2)

yearly.columns = ["Avg Cash %", "T-Bill Rate %", "Cash Contrib %", "Original Ret %", "Adjusted Ret %"]

print(yearly.to_string())

# ============================================================
# CUMULATIVE IMPACT
# ============================================================
total_cash_contribution = equity["cash_return_daily"].sum()
print(f"\n  Total cumulative cash return contribution: {total_cash_contribution*100:.1f}%")

# In dollar terms (starting from $1M)
original_terminal = equity["portfolio_value"].iloc[-1]
adjusted_terminal = equity["portfolio_with_cash"].iloc[-1]
print(f"\n  Terminal value (original):  ${original_terminal:,.0f}")
print(f"  Terminal value (with cash): ${adjusted_terminal:,.0f}")
print(f"  Difference:                 ${adjusted_terminal - original_terminal:,.0f}")

# ============================================================
# SAVE ADJUSTED EQUITY CURVE
# ============================================================
output_cols = ["date", "portfolio_value", "portfolio_with_cash", "cash", "cash_pct", 
               "strat_ret", "strat_ret_with_cash", "tbill_daily"]
equity[output_cols].to_csv("./13-trading_output_regression_insp500_spyfilter_cap15/equity_curve_with_cash_returns.csv", index=False)
print(f"\n  Saved adjusted equity curve to: equity_curve_with_cash_returns.csv")


## What To Expect

# Given your:
# - ~50% average cash allocation
# - Mix of high-rate (1999-2000: ~5%, 2023-2024: ~5%) and ZIRP (2009-2021: ~0%) periods

# Rough estimate:
# ```
# Average cash allocation:     50%
# Average T-Bill rate:         ~2% (weighted by your cash holdings)
# Expected boost:              50% × 2% = ~1% annually

# Original CAGR:               16.1%
# Adjusted CAGR:               ~17.1%

Loaded T-Bill data from FRED
Date range: 1999-01-04 00:00:00 to 2026-01-02 00:00:00
Average annual rate: 1.99%

 CASH RETURN IMPACT ANALYSIS

  Original CAGR (0% on cash):     16.43%
  Adjusted CAGR (T-Bill on cash): 17.57%
  Cash Return Boost:              +1.14%

  Original Sharpe:                1.239
  Adjusted Sharpe:                1.314

------------------------------------------------------------
 YEARLY CASH RETURN CONTRIBUTION
------------------------------------------------------------
      Avg Cash %  T-Bill Rate %  Cash Contrib %  Original Ret %  Adjusted Ret %
year                                                                           
1999        0.45           4.64            2.08           59.36           62.70
2000        0.41           5.82            2.40           37.04           40.37
2001        1.00           3.39            3.33           -0.43            2.95
2002        0.94           1.60            1.49           -2.17           -0.70
2003        0.61  